1) Reproyeccion de las capas a el CRS correspondiente de cada parque (56 capas, 8 indices y 7 métricas)
Los indices considerados son: NDVI, EVI, NDSI, NBR, LST, Albedo, NDWI, LSWI
Efas, resumen de la mediana interanual de percentile 5, Percentile 95, median, sd, CV, Dmin, Dmax anuales 


- Aiguestortes 32631 
- Ordesa 32630
- Peneda 32629
- Guadarrama 32630
- Picos 32629
- SierraNieves 32630
- SierraNev 32630
- Teide 32628


In [3]:
import os
from osgeo import gdal

#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide


parque = 'Teide'

def reproyectar_raster(archivo_entrada, archivo_salida, crs):
    try:
        # Abrir el raster de entrada
        raster_entrada = gdal.Open(archivo_entrada)

        # Reproyectar el raster
        gdal.Warp(archivo_salida, raster_entrada, dstSRS=crs)

        # Cerrar los datasets
        raster_entrada = None

        print(f"Reproyección aplicada a {archivo_entrada}")
    except Exception as e:
        print(f"Error al reproyectar {archivo_entrada}: {str(e)}")

# Definir la carpeta de entrada y salida

carpeta_entrada = f'C:/Users/carlo/Desktop/EFAs/{parque}'
carpeta_salida = f'C:/Users/carlo/Desktop/EFAs/{parque}/reprojected'

# Definir la proyección de destino (EPSG 25830)
crs = 'EPSG:25830'

# Verificar si la carpeta de salida existe y crearla si no
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)
    print(f"Carpeta de salida creada en {carpeta_salida}")

# Iterar sobre cada archivo raster en la carpeta de entrada
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.tif'):
        # Construir las rutas de archivo de entrada y salida
        archivo_entrada = os.path.join(carpeta_entrada, archivo)
        archivo_salida = os.path.join(carpeta_salida, archivo)
        
        # Reproyectar el archivo raster
        reproyectar_raster(archivo_entrada, archivo_salida, crs)

print("Proceso completado.")

Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_DMi__2018-2022_Season1-12_InterAnnualMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_DMx__2018-2022_Season1-12_InterAnnualMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_Median_2018-2022_Season1-12_InterAnnualMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_Percentile5__2018-2022_Season1-12_InterAnnualMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_Percentile95__2018-2022_Season1-12_InterAnnualSMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_sCV__2018-2022_Season1-12_InterAnnualMedian_EPSG32628_Pixel10.tif
Reproyección aplicada a C:/Users/carlo/Desktop/EFAs/Teide\albedo_SR_SENTINEL_sSD__2018-2022_Season1-12_InterAnnualMedian_E

Recortar y alinear las capas de resultados luego de hacer los calculos de la mediana y sd con el script 2 de R. 

In [6]:
import os
import glob
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
# from rasterio.enums import Resampling
from tqdm import tqdm
import shutil

# Función para copiar la estructura de carpetas de entrada a salida
def copy_folder_structure(input_folder, output_folder):
    for dirpath, dirnames, filenames in os.walk(input_folder):
        structure = os.path.join(output_folder, os.path.relpath(dirpath, input_folder))
        os.makedirs(structure, exist_ok=True)

# Obtener el directorio actual del script
base_path = os.getcwd()

# Definir parques
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# Ruta del directorio de entrada y salida
input_base_folder = "results"
output_base_folder = "results/Alineados"

# Ruta de la capa base
base_raster_path = "H:/.shortcut-targets-by-id/1M44eQfgiKNtbHWnOemX3695BodORG2B6/LW_SEM/WP4/EarthCul/Variables_layers/BaseLayers/{parque}/Slope.tif"
# base_raster_path = "C:/Users/carlo/Desktop/Diversity/grid/{parque}/Grid_new.tif"

for parque in parques:
    input_folder = os.path.join(base_path, input_base_folder, parque)
    output_folder = os.path.join(base_path, output_base_folder, parque)

    # Copiar la estructura de carpetas de entrada a salida
    copy_folder_structure(input_folder, output_folder)

    # Ruta de la capa base para este parque
    base_raster_path_parque = base_raster_path.format(parque=parque)

    with rasterio.open(base_raster_path_parque) as base_src:
        for dirpath, dirnames, filenames in os.walk(input_folder):
            for filename in filenames:
                # Verificar si el archivo es un archivo TIF
                if filename.endswith('.tif'):
                    input_file = os.path.join(dirpath, filename)
                    output_file = os.path.join(output_folder, os.path.relpath(input_file, input_folder))

                    with rasterio.open(input_file) as src:
                        data = src.read(1)
                        profile = src.profile

                        # Reproject/resample the input raster to match the base raster
                        data_resampled = np.zeros_like(base_src.read(1))
                        rasterio.warp.reproject(
                            source=data,
                            destination=data_resampled,
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=base_src.transform,
                            dst_crs=base_src.crs,
                            resampling=Resampling.nearest
                        )

                        # Update the profile of the input raster to match the base raster's metadata
                        profile_resampled = src.profile.copy()
                        profile_resampled.update({
                            'transform': base_src.transform,
                            'height': data_resampled.shape[0],
                            'width': data_resampled.shape[1],
                            'crs': base_src.crs
                        })

                        # Save
                        with rasterio.open(output_file, 'w', **profile_resampled) as dst:
                            dst.write(data_resampled, 1)

In [7]:
import os
from osgeo import gdal

# Definir la lista de parques
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# Ruta al shapefile de máscara
base_mask_folder = 'H:/.shortcut-targets-by-id/1M44eQfgiKNtbHWnOemX3695BodORG2B6/LW_SEM/WP4/EarthCul/Variables_layers/BaseLayers'

# Carpeta base para los rasters de entrada y salida
base_input_folder = "C:/Users/carlo/Desktop/EFAs/results/Alineados"
base_output_folder = "C:/Users/carlo/Desktop/EFAs/results/Clip"

# Función para copiar la estructura de carpetas de entrada a salida
def copy_folder_structure(input_folder, output_folder):
    for dirpath, dirnames, filenames in os.walk(input_folder):
        structure = os.path.join(output_folder, os.path.relpath(dirpath, input_folder))
        os.makedirs(structure, exist_ok=True)

# Iterar sobre los parques
for parque in parques:
    input_parque_folder = os.path.join(base_input_folder, parque)
    output_parque_folder = os.path.join(base_output_folder, parque)
    mask_shapefile = os.path.join(base_mask_folder, parque, 'AOI_3035.shp')

    # Crear la carpeta de salida del parque si no existe
    os.makedirs(output_parque_folder, exist_ok=True)

    # Copiar la estructura de carpetas del parque de entrada a salida
    copy_folder_structure(input_parque_folder, output_parque_folder)

    # Función para recortar rasters
    def recortar_raster(input_raster, output_raster):
        gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)

    # Iterar sobre todos los archivos en la carpeta de entrada del parque
    for dirpath, dirnames, filenames in os.walk(input_parque_folder):
        for filename in filenames:
            if filename.endswith('.tif'):
                input_raster = os.path.join(dirpath, filename)
                output_raster = os.path.join(output_parque_folder, os.path.relpath(input_raster, input_parque_folder))
                
                # Recortar el raster usando gdal.Warp
                recortar_raster(input_raster, output_raster)


gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002B596988690> >